2. Sistema de clasificación de subreddit. 

Se deberá implementar una función ```classify_subreddit(text)``` que clasifique un texto de entrada en una de las siguientes 
categorías: **MachineLearning, datascience, statistics, learnmachinelearning, computerscience, AskStatistics, artificial, analytics, datasets, deeplearning, rstats, computervision, DataScienceJobs, MLQuestions, dataengineering, data, dataanalysis, 
datascienceproject, Kaggle**.

Para ello, se proporciona un dataset sobre el que se podrán entrenar distintos algoritmos
de clasificación. La etiqueta del subreddit correspondiente se encuentra en la columna "subreddit". Se deberán probar, al menos, los siguientes 3 métodos:
- Un método basado en TF-IDF + algoritmo de clasificación de machine learning
- Un método basado en entidades reconocidas (Named-Entity Recognition) + algoritmo declasificación de machine learning 
- Un método basado en Word Embeddings + algoritmo de clasificación de machine learning

Para evaluar cada método, se utilizará la métrica f1 score, y se utilizará un 70% de los datos
del dataset para entrenamiento y un 30% para test (realizando un sampling aleatorio
previo)."""

from IPython.display import Markdown
display(Markdown(ejercicio))

Los posts de entrenamiento se encuentran en la columna 'clean_post' del dataframe, y las labels en la columna 'subreddit'. Las siguientes son las categorías en las que clasificaremos cada post:

In [2]:
categories = ['MachineLearning', 'datascience', 'statistics', 'learnmachinelearning', 'computerscience', 'AskStatistics', 'artificial', 'analytics', 'datasets', 'deeplearning', 'rstats', 'computervision', 'DataScienceJobs', 'MLQuestions', 'dataengineering', 'data', 'dataanalysis', 'datascienceproject', 'Kaggle']

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

reddit_df = pd.read_csv('processed_dataset.csv', sep=';', encoding='UTF-8')

# Tomamos el 10% de los datos de manera aleatoria
reduced_df = reddit_df.sample(frac=0.1, random_state=42)

# Dividimos en entrenamiento (70%) y prueba (30%)
# Asumimos que la columna de texto es 'clean_post' y la de sentimientos es 'sentiment'
X_train, X_test, y_train, y_test = train_test_split(
    reduced_df['clean_post'],
    reduced_df['sentiment'],
    train_size=0.7,
    random_state=42
)

### Método basado en TF-IDF + algoritmo de clasificación de machine learning

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

# (1) Ajustar TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(
    ngram_range=(1,2),
    min_df=5,
    max_df=0.7,
    sublinear_tf=True,
    stop_words='english'  # si aplica
)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf  = tfidf_vectorizer.transform(X_test)

# (2) Probar Naive Bayes con alpha personalizado
clf_tfidf_nb = MultinomialNB(alpha=0.5)
clf_tfidf_nb.fit(X_train_tfidf, y_train)

y_pred_nb = clf_tfidf_nb.predict(X_test_tfidf)
f1_nb = f1_score(y_test, y_pred_nb, average='weighted')
print(f"Método TF-IDF + Naive Bayes (alpha=0.5) - F1 Score: {f1_nb:.4f}")

# (3) Probar LogisticRegression (opcional)
clf_tfidf_lr = LogisticRegression(max_iter=200)
clf_tfidf_lr.fit(X_train_tfidf, y_train)

y_pred_lr = clf_tfidf_lr.predict(X_test_tfidf)
f1_lr = f1_score(y_test, y_pred_lr, average='weighted')
print(f"Método TF-IDF + Logistic Regression - F1 Score: {f1_lr:.4f}")


Método TF-IDF + Naive Bayes - F1 Score: 0.2213


### Un método basado en entidades reconocidas (Named-Entity Recognition) + algoritmo declasificación de machine learning 

In [ ]:
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from typing import List, Tuple

def extract_ner_features(doc):
    entity_counts = {}
    for ent in doc.ents:
        ent_label = ent.label_
        entity_counts[ent_label] = entity_counts.get(ent_label, 0) + 1
    return entity_counts

def ner_features_from_texts(texts):
    features_list = []


### Un método basado en Word Embeddings + algoritmo de clasificación de machine learning

Para evitar incompatibilidades, creamos un dataframe aparte solo con las columnas 'clean_post' y 'subreddit', para posteriormente leerlo en el entrenamiento con Word2Vec de forma más cómo, se guardará 'reddit_posts_reduced'

In [9]:
df_reduced = reddit_df[['clean_post', 'subreddit']]
df_reduced.to_csv('reddit_posts_reduced.csv', encoding='UTF-8', sep=';', index=False)